# Quantitative Momentum Strategy
## Library Imports

In [1]:
import numpy as np
import pandas as pd
import math
import requests
from scipy import stats
import xlsxwriter

## Importing List of Stocks

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making First API Call

In [3]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
#data

In [4]:
data['year1ChangePercent']

0.8391856147914228

## Executing A Batch API Call & Building DataFrame

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#    print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

##create a blank DataFrame and given data to the data frame one-by-one.

In [6]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batch_api_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
        pd.Series(
        [
            symbol,
            data[symbol]['price'],
            data[symbol]['stats']['year1ChangePercent'],
            'N/A'
        ],
        index = my_columns    
        ),
        ignore_index = True  
        )

final_dataframe        

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,124.45,0.737685,N/A
1,AAL,22.10,0.48066,N/A
2,AAP,175.98,0.339098,N/A
3,AAPL,123.10,0.855353,N/A
4,ABBV,112.04,0.32917,N/A
...,...,...,...,...
500,YUM,102.69,0.25483,N/A
501,ZBH,158.50,0.426662,N/A
502,ZBRA,489.86,1.54331,N/A
503,ZION,57.34,0.924107,N/A


## Removing Low-Momentum Stocks


In [7]:
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_dataframe = final_dataframe[:50]
final_dataframe = final_dataframe.reset_index()
final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,467,VIAC,81.620,3.36068,N/A
1,179,FCX,35.010,3.15738,N/A
2,410,SIVB,513.930,2.48541,N/A
3,317,MRO,13.000,2.41421,N/A
4,78,CARR,37.660,2.1401,N/A
5,175,FANG,83.190,2.12331,N/A
6,462,URI,296.830,2.04409,N/A
7,148,DVN,24.550,2.02637,N/A
8,212,HAL,24.580,1.98717,N/A
9,135,DISCA,65.620,1.71078,N/A


## Calculating the Number of Shares to Buy

In [8]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input('Enter the size of your portfolio:')
    
    try:
        float(portfolio_size)
    except ValueError:
        print('That is not a number! Please try again:')
        portfolio_size = input('Enter the size of your portfolio:')

portfolio_input()
print(portfolio_size)

Enter the size of your portfolio:1000000000
1000000000


In [9]:
position_size = float(portfolio_size)/len(final_dataframe.index)
for i in range(len(final_dataframe)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i, 'Price'])
final_dataframe    
  

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,467,VIAC,81.620,3.36068,245037
1,179,FCX,35.010,3.15738,571265
2,410,SIVB,513.930,2.48541,38915
3,317,MRO,13.000,2.41421,1538461
4,78,CARR,37.660,2.1401,531067
5,175,FANG,83.190,2.12331,240413
6,462,URI,296.830,2.04409,67378
7,148,DVN,24.550,2.02637,814663
8,212,HAL,24.580,1.98717,813669
9,135,DISCA,65.620,1.71078,304785


## Building a Better (and More Realistic) Momentum Strategy

In [10]:
hqm_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile',
    'HQM Score'
]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
    batch_api_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
           pd.Series(
           [
                symbol,
                data[symbol]['price'],
                'N/A',
                data[symbol]['stats']['year1ChangePercent'],
                'N/A',
                data[symbol]['stats']['month6ChangePercent'],
                'N/A',
                data[symbol]['stats']['month3ChangePercent'],
                'N/A',
                data[symbol]['stats']['month1ChangePercent'],
                'N/A',
                'N/A'
           ],
           index = hqm_columns    
           ),
           ignore_index = True 
        )
hqm_dataframe       

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,124.150,N/A,0.712002,N/A,0.228471,N/A,0.0321431,N/A,-0.0282263,N/A,N/A
1,AAL,21.800,N/A,0.48047,N/A,0.671727,N/A,0.266573,N/A,0.248698,N/A,N/A
2,AAP,178.100,N/A,0.342195,N/A,0.12574,N/A,0.100516,N/A,0.0781021,N/A,N/A
3,AAPL,121.607,N/A,0.867806,N/A,0.0363887,N/A,-0.0043324,N/A,-0.112019,N/A,N/A
4,ABBV,110.350,N/A,0.326663,N/A,0.201508,N/A,-0.00588813,N/A,0.0135533,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,104.470,N/A,0.262014,N/A,0.110025,N/A,-0.0324127,N/A,0.0005149,N/A,N/A
501,ZBH,159.570,N/A,0.425676,N/A,0.122354,N/A,0.0664391,N/A,-0.0160588,N/A,N/A
502,ZBRA,474.840,N/A,1.5407,N/A,0.858417,N/A,0.252705,N/A,0.103285,N/A,N/A
503,ZION,59.100,N/A,0.923657,N/A,0.789184,N/A,0.339491,N/A,0.150333,N/A,N/A


## Calculating Momentum Percentiles

In [11]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
               ]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        if hqm_dataframe.loc[row, f'{time_period} Price Return'] == None:
            hqm_dataframe.loc[row, f'{time_period} Price Return'] = 0

score = stats.percentileofscore            
for row in hqm_dataframe.index:
    for time_period in time_periods:
    
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'

        hqm_dataframe.loc[row, percentile_col] = score(hqm_dataframe[change_col], hqm_dataframe.loc[row, change_col])/100            
#hqm_dataframe.loc[row, f'{time_period} Return Percentile'] = stats.percentileofscore(hqm_dataframe[f'{time_period} Price Return'], hqm_dataframe.loc[row, f'{time_period} Price Return'])

hqm_dataframe        

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,124.150,N/A,0.712002,0.728713,0.228471,0.534653,0.0321431,0.380198,-0.0282263,0.231683,N/A
1,AAL,21.800,N/A,0.48047,0.544554,0.671727,0.883168,0.266573,0.867327,0.248698,0.972277,N/A
2,AAP,178.100,N/A,0.342195,0.409901,0.12574,0.364356,0.100516,0.544554,0.0781021,0.69703,N/A
3,AAPL,121.607,N/A,0.867806,0.823762,0.0363887,0.215842,-0.0043324,0.267327,-0.112019,0.0653465,N/A
4,ABBV,110.350,N/A,0.326663,0.392079,0.201508,0.485149,-0.00588813,0.255446,0.0135533,0.457426,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,104.470,N/A,0.262014,0.328713,0.110025,0.334653,-0.0324127,0.182178,0.0005149,0.39802,N/A
501,ZBH,159.570,N/A,0.425676,0.493069,0.122354,0.358416,0.0664391,0.461386,-0.0160588,0.283168,N/A
502,ZBRA,474.840,N/A,1.5407,0.972277,0.858417,0.950495,0.252705,0.851485,0.103285,0.784158,N/A
503,ZION,59.100,N/A,0.923657,0.845545,0.789184,0.932673,0.339491,0.928713,0.150333,0.89703,N/A


## Calculating the high-quality momentum(HQM) Score

In [12]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)

## Selecting the 50 Best Momentum Stocks

In [13]:
hqm_dataframe.sort_values('HQM Score', ascending = False, inplace = True)
hqm_dataframe = hqm_dataframe[:50]
hqm_dataframe.reset_index(drop = True, inplace = True)
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,VIAC,79.730,N/A,3.38114,1,1.84843,0.99802,1.23613,1,0.458067,0.99802,0.99901
1,DISCA,64.890,N/A,1.74125,0.982178,1.80848,0.99604,1.22949,0.99802,0.469675,1,0.994059
2,MRO,12.000,N/A,2.34843,0.994059,1.72447,0.994059,0.699807,0.992079,0.355776,0.994059,0.993564
3,DVN,24.420,N/A,2.09555,0.988119,1.56155,0.984158,0.53429,0.990099,0.271312,0.980198,0.985644
4,DISCK,54.850,N/A,1.38708,0.960396,1.5801,0.986139,1.12915,0.99604,0.429356,0.99604,0.984653
5,FANG,84.110,N/A,2.21163,0.992079,1.5867,0.988119,0.744787,0.994059,0.222183,0.962376,0.984158
6,OXY,28.780,N/A,1.33273,0.954455,1.6192,0.990099,0.478339,0.986139,0.166493,0.920792,0.962871
7,TPR,44.620,N/A,1.35768,0.956436,2.03027,1,0.480076,0.988119,0.132472,0.867327,0.95297
8,GPS,29.240,N/A,1.48959,0.970297,0.665335,0.877228,0.3398,0.930693,0.330775,0.992079,0.942574
9,EOG,76.190,N/A,1.02426,0.887129,0.787951,0.930693,0.400236,0.966337,0.275017,0.982178,0.941584


## Calculating the Number of Shares to Buy

In [14]:
portfolio_input()

Enter the size of your portfolio:100000000


In [15]:
position_size = float(portfolio_size)/len(hqm_dataframe.index)
for row in hqm_dataframe.index:
    hqm_dataframe.loc[row, 'Number of Shares to Buy'] = math.floor(position_size/hqm_dataframe.loc[row, 'Price'])
   
hqm_dataframe


C:\Users\PRATHAM\anaconda3\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,VIAC,79.730,25084,3.38114,1,1.84843,0.99802,1.23613,1,0.458067,0.99802,0.99901
1,DISCA,64.890,30821,1.74125,0.982178,1.80848,0.99604,1.22949,0.99802,0.469675,1,0.994059
2,MRO,12.000,166666,2.34843,0.994059,1.72447,0.994059,0.699807,0.992079,0.355776,0.994059,0.993564
3,DVN,24.420,81900,2.09555,0.988119,1.56155,0.984158,0.53429,0.990099,0.271312,0.980198,0.985644
4,DISCK,54.850,36463,1.38708,0.960396,1.5801,0.986139,1.12915,0.99604,0.429356,0.99604,0.984653
5,FANG,84.110,23778,2.21163,0.992079,1.5867,0.988119,0.744787,0.994059,0.222183,0.962376,0.984158
6,OXY,28.780,69492,1.33273,0.954455,1.6192,0.990099,0.478339,0.986139,0.166493,0.920792,0.962871
7,TPR,44.620,44822,1.35768,0.956436,2.03027,1,0.480076,0.988119,0.132472,0.867327,0.95297
8,GPS,29.240,68399,1.48959,0.970297,0.665335,0.877228,0.3398,0.930693,0.330775,0.992079,0.942574
9,EOG,76.190,26250,1.02426,0.887129,0.787951,0.930693,0.400236,0.966337,0.275017,0.982178,0.941584


## Formatting Our Excel Output

In [16]:
writer = pd.ExcelWriter('moment_stretgy.xlsx', engine = 'xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name = 'Momentum Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File

In [17]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [18]:
column_formats = {
    'A': ['Ticker', string_template],
    'B': ['Price', dollar_template],
    'C': ['Number of Shares to Buy', integer_template],
    'D': ['One-Year Price Return', percent_template],
    'E': ['One-Year Return Percentile', percent_template],
    'F': ['Six-Month Price Return', percent_template],
    'G': ['Six-Month Return Percentile', percent_template],
    'H': ['Three-Month Price Return', percent_template],
    'I': ['Three-Month Return Percentile', percent_template],
    'J': ['One-Month Price Return', percent_template],
    'K': ['One-Month Return Percentile', percent_template],
    'L': ['HQM Score', percent_template]
}

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 22, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1',column_formats[column][0], column_formats[column][1])

## Saving Our Excel Output

In [19]:
writer.save()